In [39]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

import Base: show

In [40]:
function createSubgraph(model, index::Int)
    noise_add = add!(model, AdditionNode())
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    yn    = add!(model, datavar(:yn, Float64))
    
    connect!(noise_add, :in2, noise, 1)
    connect!(noise_add, :out, yn, 1)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    model = Model(DefaultMessageGate())
    
    c0_add = add!(model, AdditionNode())
    
    c0       = add!(model, constvar(:c0, 1.0))
    x0_prior = add!(model, datavar(:x0_prior, Normal{Float64}))
    
    connect!(c0_add, :in1, x0_prior, 1)
    connect!(c0_add, :in2, c0, 1)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(model, index)
        xn            = add!(model, randomvar(:xn, 3))
        
        c_add = add!(model, AdditionNode())
        cn    = add!(model, constvar(:cn, 1.0))
        
        connect!(prev_c_add, :out, xn, 1)
        connect!(noise_add, :in1, xn, 2)
        connect!(c_add, :in1, xn, 3)
        connect!(c_add, :in2, cn, 1)
        
        activate!(model, prev_c_add)
        activate!(model, noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = add!(model, AdditionNode());
    
    x_last     = add!(model, randomvar(:x_last, 2))
    y_last     = add!(model, datavar(:ylast, Float64))
    last_noise = add!(model, constvar(:last_noise, Normal(0.0, sqrt(200.0))))
    
    connect!(prev_c_add, :out, x_last, 1)
    connect!(last_noise_add, :in1, x_last, 2)
    connect!(last_noise_add, :in2, last_noise, 1)
    connect!(last_noise_add, :out, y_last, 1)
    
    activate!(model, prev_c_add)
    activate!(model, last_noise_add)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [41]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    marginals     = Vector{Normal{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    @inbounds for (index, x) in enumerate(xs)
        subscriptions[index] = subscribe!(getmarginal(x), (d) -> marginals[index] = getdata(d))
    end
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

smoothing (generic function with 1 method)

In [42]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [45]:
@btime smoothing($data)

  56.254 ms (406673 allocations: 14.84 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.8006436493919095, σ=0.5773406469256954)
 Normal{Float64}(μ=1.8006436493919094, σ=0.5773406469256954)
 Normal{Float64}(μ=2.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=3.8006436493919082, σ=0.5773406469256953)
 Normal{Float64}(μ=4.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=5.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=6.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=7.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=8.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=9.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=10.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=11.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=12.800643649391908, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=588.8006436493908, σ=0.577340646925696)
 Normal{Float64}(μ=589.8006436493907, σ=0.577340646925696)
 Normal{Float64}(μ=590.8006436493908, σ=0.577340646925696

In [44]:
@time smoothing(data)

  0.074483 seconds (406.67 k allocations: 14.842 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.8006436493919095, σ=0.5773406469256954)
 Normal{Float64}(μ=1.8006436493919094, σ=0.5773406469256954)
 Normal{Float64}(μ=2.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=3.8006436493919082, σ=0.5773406469256953)
 Normal{Float64}(μ=4.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=5.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=6.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=7.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=8.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=9.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=10.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=11.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=12.800643649391908, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=588.8006436493908, σ=0.577340646925696)
 Normal{Float64}(μ=589.8006436493907, σ=0.577340646925696)
 Normal{Float64}(μ=590.8006436493908, σ=0.577340646925696

In [43]:
@time smoothing(data)

  0.385372 seconds (1.14 M allocations: 50.514 MiB, 3.14% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.8006436493919095, σ=0.5773406469256954)
 Normal{Float64}(μ=1.8006436493919094, σ=0.5773406469256954)
 Normal{Float64}(μ=2.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=3.8006436493919082, σ=0.5773406469256953)
 Normal{Float64}(μ=4.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=5.800643649391908, σ=0.5773406469256954)
 Normal{Float64}(μ=6.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=7.800643649391909, σ=0.5773406469256954)
 Normal{Float64}(μ=8.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=9.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=10.800643649391912, σ=0.5773406469256954)
 Normal{Float64}(μ=11.80064364939191, σ=0.5773406469256953)
 Normal{Float64}(μ=12.800643649391908, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=588.8006436493908, σ=0.577340646925696)
 Normal{Float64}(μ=589.8006436493907, σ=0.577340646925696)
 Normal{Float64}(μ=590.8006436493908, σ=0.577340646925696

In [29]:
n = AdditionNode()

FactorNode{typeof(+),3,Tuple{Tuple{Int64,Int64,Int64}},ReactiveMP.FactorNodeLocalMarginals{1}}(+, (in1, in2, out), ((1, 2, 3),), ReactiveMP.FactorNodeLocalMarginals{1}(((:in1in2out, Base.RefValue{Union{Nothing, Marginal}}(nothing)),)))

In [38]:
@btime $n.marginals[:in1in2out]

  32.157 ns (0 allocations: 0 bytes)


In [22]:
variables = (:x, :y, :z)
variables2 = map(v -> varnode(v), variables)

(x, y, z)

In [23]:
factorisation = ((1, 2, 3), )

((1, 2, 3),)

In [10]:
names = map(q -> reduce((r, v) -> Symbol(r, variables[v]), q, init = Symbol()), factorisation)
types = Tuple{ map(n -> Union{Nothing, Marginal}, names)... }
init  = map(n -> nothing, names)

(nothing, nothing)

In [11]:
@btime NamedTuple{$names, $types}($init)

  159.724 ns (1 allocation: 32 bytes)


NamedTuple{(:x, :yz),Tuple{Union{Nothing, Marginal},Union{Nothing, Marginal}}}((nothing, nothing))

In [18]:
function preallocate_marginals(variables::NTuple{N, VariableNode}, factorisation) where N
    names = map(q -> reduce((r, v) -> Symbol(r, variables[v]), q, init = Symbol()), factorisation)
    init  = map(n -> (n, Ref{Union{Nothing, Marginal}}(nothing)), names)
    return NTuple{length(factorisation), Tuple{Symbol, Ref{Union{Nothing, Marginal}}}}(init)
end

function preallocate_marginals2(variables::NTuple{N, VariableNode}, factorisation) where N
    names = map(n -> Symbol(n...), map(q -> map(v -> name(variables[v]), q), factorisation))
    init  = map(n -> (n, Ref{Union{Nothing, Marginal}}(nothing)), names)
    return NTuple{length(factorisation), Tuple{Symbol, Ref{Union{Nothing, Marginal}}}}(init)
end

function preallocate_marginals3(variables::NTuple{N, VariableNode}, factorisation) where N
    names = map(n -> Symbol(n...), map(q -> map(v -> name(variables[v]), q), factorisation))
    types = Tuple{ map(n -> Ref{Union{Nothing, Marginal}}, names)... }
    init  = map(n -> Ref{Union{Nothing, Marginal}}(nothing), names)
    return NamedTuple{names, types}(init)
end

preallocate_marginals3 (generic function with 1 method)

In [28]:
@btime ReactiveMP.FactorNodeLocalMarginals($variables, $factorisation)

  160.147 ns (7 allocations: 256 bytes)


ReactiveMP.FactorNodeLocalMarginals{1}(((:xyz, Base.RefValue{Union{Nothing, Marginal}}(nothing)),))

In [121]:
function namesonly(variables, factorisation)
    return map(q -> reduce((r, v) -> Symbol(r, name(variables[v])), q, init = Symbol()), factorisation) 
end

function namesonly2(variables, factorisation)
    return map(n -> Symbol(n...), map(q -> map(v -> name(variables[v]), q), factorisation))
end

namesonly2 (generic function with 1 method)

In [123]:
@btime namesonly2($variables2, $factorisation)

  134.808 ns (4 allocations: 208 bytes)


(:x, :yz)

In [114]:
@btime namesonly($variables2, $factorisation)

  391.995 ns (12 allocations: 592 bytes)


(:x, :yz)

In [143]:
p3 = preallocate_marginals3(variables2, factorisation)

NamedTuple{(:x, :yz),Tuple{Ref{Union{Nothing, Marginal}},Ref{Union{Nothing, Marginal}}}}((Base.RefValue{Union{Nothing, Marginal}}(nothing), Base.RefValue{Union{Nothing, Marginal}}(nothing)))

In [147]:
@btime $p3[:yz][] = Marginal(1)

  2.218 ns (0 allocations: 0 bytes)


Marginal{Int64}(1)

In [145]:
@btime preallocate_marginals($variables2, $factorisation)
@btime preallocate_marginals2($variables2, $factorisation)
@btime preallocate_marginals3($variables2, $factorisation)

  556.688 ns (16 allocations: 688 bytes)
  150.992 ns (8 allocations: 304 bytes)
  561.119 ns (8 allocations: 304 bytes)


NamedTuple{(:x, :yz),Tuple{Ref{Union{Nothing, Marginal}},Ref{Union{Nothing, Marginal}}}}((Base.RefValue{Union{Nothing, Marginal}}(nothing), Base.RefValue{Union{Nothing, Marginal}}(nothing)))

In [24]:
p = preallocate_marginals2(variables2, factorisation)

((:xyz, Base.RefValue{Union{Nothing, Marginal}}(nothing)),)

In [26]:
@btime $p[findnext(d -> d[1] === :xyz, $p, 1)][2]

  2.460 ns (0 allocations: 0 bytes)


Base.RefValue{Union{Nothing, Marginal}}(nothing)

In [103]:
function getpreallocatedmarginal(preallocated, name)
    return preallocated[findnext(d -> d[1] === name, preallocated, 1)][2]
end

getpreallocatedmarginal (generic function with 1 method)

In [109]:
getpreallocatedmarginal(p, :yz)[] = Marginal(1)

Marginal{Int64}(1)

In [110]:
p

((:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)), (:yz, Base.RefValue{Union{Nothing, Marginal}}(Marginal{Int64}(1))))

In [69]:
d = Dict{Symbol, Marginal}()
@btime $d[:x] = Marginal(1)

  15.583 ns (1 allocation: 16 bytes)


Marginal{Int64}(1)

In [87]:
@btime NTuple{3, Tuple{Symbol, Ref{Union{Nothing, Marginal}}}}(
    (
        Tuple{Symbol, Ref{Union{Nothing, Marginal}}}((:x, Ref{Union{Nothing, Marginal}}(nothing))),
        Tuple{Symbol, Ref{Union{Nothing, Marginal}}}((:x, Ref{Union{Nothing, Marginal}}(nothing))),
        Tuple{Symbol, Ref{Union{Nothing, Marginal}}}((:x, Ref{Union{Nothing, Marginal}}(nothing)))
    )
)

  37.687 ns (7 allocations: 176 bytes)


((:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)), (:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)), (:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)))

In [97]:
preallocate_marginals(variables2, factorisation)

MethodError: MethodError: Cannot `convert` an object of type Base.RefValue{Union{Nothing, Marginal}} to an object of type Tuple{Symbol,Ref{Union{Nothing, Marginal}}}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Tuple{Any,Vararg{Any,N} where N} at essentials.jl:309
  convert(::Type{T}, !Matched::Tuple{Any,Vararg{Any,N} where N}) where T<:Tuple{Any,Vararg{Any,N} where N} at essentials.jl:310
  convert(::Type{T}, !Matched::CartesianIndex) where T<:Tuple at multidimensional.jl:134
  ...

In [95]:
@btime $d[findnext(d -> d[1] === :z, $d, 1)][2][] = Marginal(1)

  2.388 ns (0 allocations: 0 bytes)


Marginal{Int64}(1)

In [92]:
d

((:x, Base.RefValue{Union{Nothing, Marginal}}(Marginal{Int64}(1))), (:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)), (:x, Base.RefValue{Union{Nothing, Marginal}}(nothing)))